# ResNet with TensorFlow
## An implementation of a Convolutional Neural Network using classes to classify the mnist handwritten digits database using the quickstart for experts guide on TensorFlow.

### Setup

In [1]:
# Import statements
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model, losses, models, layers, datasets
import matplotlib.pyplot as plt

TensorFlow version: 2.12.0


In [3]:
# Data
fashion_mnist = datasets.mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Scaling images between 0 and 1
x_train, x_test = x_train/255, x_test/255

# Resizing images to be 32x32 because that is the minimum
x_train = tf.pad(x_train, [[0,0], [2,2], [2,2]])
x_test = tf.pad(x_test, [[0, 0], [2,2], [2,2]])

x_train = tf.expand_dims(x_train, axis=3, name=None)
x_test = tf.expand_dims(x_test, axis=3, name=None)
x_train = tf.repeat(x_train, 3, axis=3)
x_test = tf.repeat(x_test, 3, axis=3)


# Create a validation set
x_val = x_train[-2000:,:,:]
y_val = y_train[-2000:]
x_train = x_train[:-2000,:,:]
y_train = y_train[:-2000]

# Batching and shuffling the dataset (Marked for review)
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

### Building the Model

In [4]:
# Base Model
base_model = tf.keras.applications.ResNet152(weights="imagenet", include_top = False, input_shape = (32,32,3))

for layer in base_model.layers:
    layer.trainable = False

class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.flatten = Flatten()(base_model.output)
        self.d1 = Dense(1024, activation="relu")
        self.d2 = Dense(10)

    def call(self, x):
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

model = MyModel()
predictions = model


234698864/234698864 [==============================] - 1s 0us/step


In [ ]:
# Specific Code
x = Flatten()(base_model.output)
x = Dense(1000, activation="relu")(x)
predictions = layers.Dense(10, activation = "softmax")(x)

In [ ]:
head_model = Model(inputs = base_model.input, outputs=predictions)
head_model.compile(optimizer="adam", loss=losses.sparse_categorical_crossentropy, metrics=["accuracy"])

### Running the Model

In [ ]:
history = head_model.fit(x_train, y_train, batch_size=64, epochs=40, validation_data=(x_val, y_val))

Epoch 1/40
907/907 [==============================] - 59s 45ms/step - loss: 0.8952 - accuracy: 0.7051 - val_loss: 0.5009 - val_accuracy: 0.8510
Epoch 2/40
907/907 [==============================] - 37s 41ms/step - loss: 0.5889 - accuracy: 0.8080 - val_loss: 0.3896 - val_accuracy: 0.8855
Epoch 3/40
907/907 [==============================] - 38s 42ms/step - loss: 0.5044 - accuracy: 0.8364 - val_loss: 0.3292 - val_accuracy: 0.9040
Epoch 4/40
907/907 [==============================] - 37s 41ms/step - loss: 0.4506 - accuracy: 0.8512 - val_loss: 0.3372 - val_accuracy: 0.8980
Epoch 5/40
907/907 [==============================] - 37s 41ms/step - loss: 0.4214 - accuracy: 0.8610 - val_loss: 0.2806 - val_accuracy: 0.9190
Epoch 6/40
907/907 [==============================] - 38s 42ms/step - loss: 0.3915 - accuracy: 0.8724 - val_loss: 0.2664 - val_accuracy: 0.9255
Epoch 7/40
907/907 [==============================] - 37s 41ms/step - loss: 0.3649 - accuracy: 0.8801 - val_loss: 0.2274 - val_accuracy:

KeyboardInterrupt: ignored

In [ ]:
head_model.evaluate(x_test, y_test)

313/313 [==============================] - 18s 55ms/step - loss: 0.2853 - accuracy: 0.9071


[0.2853456139564514, 0.9071000218391418]

In [ ]:
head_model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 38, 38, 3)    0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 16, 16, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 16, 16, 64)   256         ['conv1_conv[0][0]']             
                                                                                              